# Torch
## Check GPU¶

In [1]:
#from apex import amp,optimizers

In [2]:
import torch
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(1))

TITAN Xp


## Set torch default parameters¶

In [3]:
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=8)
torch.backends.cudnn.benchmark = True

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/train_cnn_gp_torch')

# Set Arguments

In [4]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()

'''Training Parameters'''
parser.add_argument('--batch_size', type=int, default=300, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200, help='number of epochs')
parser.add_argument('--grad_clip', type=float, default=5., help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.0014, help='learning rate')
parser.add_argument('--learning_rate_clip', type=float, default=0.0000001, help='learning rate clip')
parser.add_argument('--decay_rate', type=float, default=.8, help='decay rate for rmsprop')
parser.add_argument('--weight_decay', type=float, default=.0001, help='decay rate for rmsprop')
parser.add_argument('--batch_norm_decay', type=float, default=.999, help='decay rate for rmsprop')
parser.add_argument('--keep_prob', type=float, default=1.0, help='dropout keep probability')
parser.add_argument('--lamda_weights', type=float, default=.01, help='lamda weight')
parser.add_argument('--data_argumentation', type=bool, default=True, help='whether do data argument')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data nomalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')
parser.add_argument('--output_dim', default=3, type=int, help='output dimention.')
parser.add_argument('--feat_dim', default=128, type=int, help='feature dimention.')

'''Configure'''
parser.add_argument('--network', type=str, default='vggnet_localization')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/gp_net_torch', help='rnn, gru, or lstm')

'''
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08',
                                                            '/notebooks/michigan_nn_data/2012_01_15',
                                                            '/notebooks/michigan_nn_data/2012_01_22',
                                                            '/notebooks/michigan_nn_data/2012_02_02',
                                                            '/notebooks/michigan_nn_data/2012_02_04',
                                                            '/notebooks/michigan_nn_data/2012_02_05',
                                                            '/notebooks/michigan_nn_data/2012_03_31',
                                                            '/notebooks/michigan_nn_data/2012_09_28'])
'''
#parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_15'])

parser.add_argument('--seed', default=1337, type=int)
parser.add_argument('--save_every', type=int, default=2000, help='save frequency')
parser.add_argument('--display', type=int, default=50, help='display frequency')

sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [5]:
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
from PIL import Image
import numpy as np
import random

#import gpflow.multioutput.kernels as mk
import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.utils import LocalizationDataset
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.train_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False)
[args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
#torch.save([args.norm_mean, args.norm_std], '/notebooks/global_localization/norm_mean_std.pt')

'''
dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=True, pin_memory=True)
'''

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'
100%|██████████| 22584/22584 [00:27<00:00, 820.12it/s]


'\ndataloader = DataLoader(dataset, batch_size=args.batch_size,                         shuffle=True, num_workers=0,                         drop_last=True, pin_memory=True)\n'

In [6]:
lengths = [round(len(dataset)*0.7), round(len(dataset)*0.3)]
training_dataset, test_dataset = random_split(dataset, lengths)

In [7]:
training_dataloader = DataLoader(training_dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=False, pin_memory=True)

test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=False, pin_memory=True)

In [8]:
args.norm_mean, args.norm_std

(tensor([-123.49977112,  416.04232788,   -8.91912651]),
 tensor([124.76653290, 191.42112732,   4.87461853]))

# Define Model

In [9]:
def normalize(target, norm_mean, norm_std):
    target_trans = target[:,:3]
    target_trans = torch.div(torch.sub(target_trans,norm_mean),norm_std)
    target_normed = torch.cat([target_trans,target[:,3:]],dim=1)
    return target_normed 

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet.resnet50(pretrained=True)
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        dense_feat = self.resnet(input_data)
        global_context_feat = self.global_context(dense_feat)
        global_output, trans_feat, rot_feat = self.global_regressor(global_context_feat)
        return global_output, trans_feat, rot_feat
    
class MultitaskGPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        # We have to mark the CholeskyVariationalDistribution as batch
        # so that we learn a variational distribution for each task
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([3])
        )

        # We have to wrap the VariationalStrategy in a MultitaskVariationalStrategy
        # so that the output will be a MultitaskMultivariateNormal rather than a batch output
        variational_strategy = gpytorch.variational.MultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=3
        )

        super().__init__(variational_strategy)

        # The mean and covariance modules should be marked as batch
        # so we learn a different set of hyperparameters
        #self.net = Model()
        #self.net.load_state_dict(torch.load(os.path.join(args.model_dir,'model-23-96000.pth')))
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([1]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([1])),
            batch_shape=torch.Size([1])
        )

    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
class GPModel(gpytorch.Module):
    def __init__(self, inducing_points):
        super(GPModel, self).__init__()
        self.net = Model()
        #self.net.load_state_dict(torch.load(os.path.join('/notebooks/global_localization/dual_resnet_torch','model-23-96000.pth')))
        self.gp = MultitaskGPModel(inducing_points)
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)

    def forward(self, x):
        global_output, trans_feat, _ = self.net(x)
        _, rot_pred = torch.split(global_output, [3, 4], dim=1)
        output = self.gp(trans_feat)
        
        return output,rot_pred
    
class FeatureModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = Model()
    
    def forward(self,x):
        _, trans_feat, _ = self.net(x)
        return trans_feat

## Creat Model

In [10]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(device)

model = FeatureModel().to(device)

# Load Resnet
state_dict = torch.load('/notebooks/global_localization/dual_resnet_torch/pretrained.pth',map_location=device)
for name,param in state_dict.items():
    print(name, param.shape)
print('Parameters layer:',len(state_dict.keys()))
model.net.load_state_dict(state_dict)

resnet.conv1.weight torch.Size([64, 1, 7, 7])
resnet.bn1.weight torch.Size([64])
resnet.bn1.bias torch.Size([64])
resnet.bn1.running_mean torch.Size([64])
resnet.bn1.running_var torch.Size([64])
resnet.bn1.num_batches_tracked torch.Size([])
resnet.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
resnet.layer1.0.bn1.weight torch.Size([64])
resnet.layer1.0.bn1.bias torch.Size([64])
resnet.layer1.0.bn1.running_mean torch.Size([64])
resnet.layer1.0.bn1.running_var torch.Size([64])
resnet.layer1.0.bn1.num_batches_tracked torch.Size([])
resnet.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
resnet.layer1.0.bn2.weight torch.Size([64])
resnet.layer1.0.bn2.bias torch.Size([64])
resnet.layer1.0.bn2.running_mean torch.Size([64])
resnet.layer1.0.bn2.running_var torch.Size([64])
resnet.layer1.0.bn2.num_batches_tracked torch.Size([])
resnet.layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
resnet.layer1.0.bn3.weight torch.Size([256])
resnet.layer1.0.bn3.bias torch.Size([256])
resnet.layer1.0.bn3.ru

<All keys matched successfully>

In [11]:
'''
state_dict = torch.load(os.path.join(args.model_dir,'pretrained.pth'),map_location=device)
for name,param in state_dict.items():
    print(name, param.shape)
print('Parameters layer:',len(state_dict.keys()))
model.load_state_dict(state_dict)
'''

"\nstate_dict = torch.load(os.path.join(args.model_dir,'pretrained.pth'),map_location=device)\nfor name,param in state_dict.items():\n    print(name, param.shape)\nprint('Parameters layer:',len(state_dict.keys()))\nmodel.load_state_dict(state_dict)\n"

In [12]:
# Disable resnet
for param in model.parameters():
    param.requires_grad = False

# Display Learn parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.shape)

# Training
## Parameters

In [13]:
args.norm_mean = args.norm_mean.to(device)
args.norm_std = args.norm_std.to(device)

np.savetxt("/notebooks/michigan_nn_data/toy/norm_mean.csv", args.norm_mean.cpu().numpy(), delimiter=",")
np.savetxt("/notebooks/michigan_nn_data/toy/norm_std.csv", args.norm_std.cpu().numpy(), delimiter=",")

## Train

In [14]:
model.eval()

x_train = 0
y_train = 0

for b, data in enumerate(training_dataloader, 0):
    start = time.time()
    with torch.no_grad():
        x,y = data.values()
        x,y = x.to(device),y.to(device)
        # normalize targets
        y = normalize(y,args.norm_mean, args.norm_std)
        trans_feat = model(x)
        
        if b == 0:
            x_train = trans_feat
            y_train = y
        else:
            x_train = torch.cat([x_train,trans_feat],dim=0)
            y_train = torch.cat([y_train,y],dim=0)
    end = time.time()
    with torch.no_grad():
        if ((b+1)%5 == 0):
            print("{}/{}, time/batch = {:.3f}".format((b+1),len(training_dataloader),end - start))

5/53, time/batch = 0.688
10/53, time/batch = 0.688
15/53, time/batch = 0.691
20/53, time/batch = 0.692
25/53, time/batch = 0.695
30/53, time/batch = 0.687
35/53, time/batch = 0.692
40/53, time/batch = 0.693
45/53, time/batch = 0.693
50/53, time/batch = 0.701


In [15]:
assert x_train.shape == torch.Size([len(training_dataset), 128])
print(x_train.shape)
print(y_train.shape)
np.savetxt("/notebooks/michigan_nn_data/toy/x_train.csv", x_train.cpu().numpy(), delimiter=",")
np.savetxt("/notebooks/michigan_nn_data/toy/y_train.csv", y_train.cpu().numpy(), delimiter=",")

torch.Size([15809, 128])
torch.Size([15809, 7])


## Test

In [16]:
model.eval()

x_test = 0
y_test = 0

for b, data in enumerate(test_dataloader, 0):
    start = time.time()
    with torch.no_grad():
        x,y = data.values()
        x,y = x.to(device),y.to(device)
        # normalize targets
        y = normalize(y,args.norm_mean, args.norm_std)
        trans_feat = model(x)
        
        if b == 0:
            x_test = trans_feat
            y_test = y
        else:
            x_test = torch.cat([x_test,trans_feat],dim=0)
            y_test = torch.cat([y_test,y],dim=0)
    end = time.time()
    with torch.no_grad():
        if ((b+1)%5 == 0):
            print("{}/{}, time/batch = {:.3f}".format((b+1),len(training_dataloader),end - start))

5/53, time/batch = 0.700
10/53, time/batch = 0.694
15/53, time/batch = 0.697
20/53, time/batch = 0.693


In [17]:
assert x_test.shape == torch.Size([len(test_dataset), 128])
print(x_test.shape)
print(y_test.shape)
np.savetxt("/notebooks/michigan_nn_data/toy/x_test.csv", x_test.cpu().numpy(), delimiter=",")
np.savetxt("/notebooks/michigan_nn_data/toy/y_test.csv", y_test.cpu().numpy(), delimiter=",")

torch.Size([6775, 128])
torch.Size([6775, 7])
